In [15]:
import brainpy as bp
import brainpy.math as bm
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5,6"  # specify which GPU(s) to be used
bm.disable_gpu_memory_preallocation()
bm.set_platform('gpu')

In [20]:
class Izh(bp.NeuGroupNS):
    def __init__(self, size, a=0.02, b=-0.1, c=-55.0, tau_ref=0., d= 6.0, V_th=30., method='rk2', **kwargs):
        super(Izh, self).__init__(size=size, **kwargs)
        # parameters
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.V_th = V_th
        self.tau_ref = tau_ref
        
        # 初始化变量
        self.V = bm.Variable(bm.zeros(self.num) - 65.)  # 膜电位
        self.u = bm.Variable(self.V * b)  # u变量
        self.input = bm.Variable(bm.zeros(self.num))  # 外界输入
        self.t_last_spike = bm.Variable(bm.ones(self.num) * -1e7)  # 上一次脉冲发放时间
        self.refractory = bm.Variable(bm.zeros(self.num, dtype=bool))  # 是否处于不应期
        self.spike = bm.Variable(bm.zeros(self.num, dtype=bool))  # 脉冲发放状态
        
        # 定义积分器
        self.integral = bp.odeint(f=self.derivative, method=method)
            
    def dV(self, V, t, u, Iext):
        return 0.04 * V * V + 5 * V + 140 - u + Iext * 100
    
    def du(self, u, t, V):
        return self.a * (self.b * V - u)
    
    # 将两个微分方程联合为一个，以便同时积分
    @property
    def derivative(self):
        return bp.JointEq([self.dV, self.du])

    def update(self):
        _t = bp.share.load('t')
        _dt = bp.share.load('dt')
        V, u = self.integral(self.V, self.u, _t, self.input, dt=_dt)  # 更新变量V, u
        refractory = (_t - self.t_last_spike) <= self.tau_ref  # 判断神经元是否处于不应期
        V = bm.where(refractory, self.V, V)  # 若处于不应期，则返回原始膜电位self.V，否则返回更新后的膜电位V
        spike = V > self.V_th  # 将大于阈值的神经元标记为发放了脉冲
        self.spike.value = spike  # 更新神经元脉冲发放状态
        self.t_last_spike.value = bm.where(spike, _t, self.t_last_spike)  # 更新最后一次脉冲发放时间
        self.V.value = bm.where(spike, self.c, V)  # 将发放了脉冲的神经元的V置为c，其余不变
        self.u.value = bm.where(spike, u + self.d, u)  # 将发放了脉冲的神经元的u增加d，其余不变
        self.refractory.value = bm.logical_or(refractory, spike)  # 更新神经元是否处于不应期
        self.input[:] = 0.  # 重置外界输入

In [21]:
s =bm.array([0.5,1,1.5])
phi = bm.array([[0.3313,0.8148,0.4364],[0.8835,0.3621,0.2182],[0.3313,0.4527,0.8729]])
b = bm.dot(phi.T,s)
w = bm.dot(phi.T,phi)
w[bm.diag_indices_from(w)] = 0
print(w)
print(b)

Array(value=Array([[0.        , 0.7397555 , 0.62646115],
                   [0.7397555 , 0.        , 0.82969487],
                   [0.62646115, 0.82969487, 0.        ]]),
      dtype=float32)
Array(value=Array([1.5460999, 1.44855  , 1.74575  ]), dtype=float32)


In [22]:
class Spiking_LCA(bp.DynamicalSystemNS):
    def __init__(self, num, w, b, lamb, method='rk2'):
        super(Spiking_LCA, self).__init__()
        
        # parameter setting
        self.lamb = lamb
        
        # network size
        num_neuron = int(num)
        self.N = Izh(num_neuron)
        self.average_current = bm.Variable(bm.zeros(num_neuron))          # 平均电流记录
        # synapses
        self.N2N = bp.synapses.Exponential(pre=self.N,
                                            post=self.N,
                                            conn= bp.connect.All2All(),
                                            g_max= w,
                                            tau=1.,
                                            output=bp.synouts.CUBA(),
                                            method=method)

    def reversed_function(self,x):
        return 0.24999688 *x**2 + 2.86516435*x + 0.20432898

    # def reversed_function(self, x):
    #     return -0.0471247 * x**3 + 0.32773097 * x**2 + 2.83983869 * x + 0.20385288

                
    def update(self):
        t = bp.share.load('t')
        self.N2N()
        self.average_current.value = self.average_current *t/(t + 1) + (b - self.N2N.g)/(t+1)
        firing_rate  =  bm.maximum(self.average_current - self.lamb, 0.)
        self.N.input =  bm.where(firing_rate > 1e-7, self.reversed_function(firing_rate) , 0)
        self.N()

In [23]:
net    = Spiking_LCA(w.shape[0],w,b,0.1)
total_period = 3000
runner = bp.DSRunner(net,monitors=['N.V','N.spike'], dt = 0.01)
runner.run(total_period)

Predict 300000 steps: :   0%|          | 638/300000 [00:00<03:26, 1447.98it/s]

Predict 300000 steps: : 100%|██████████| 300000/300000 [02:14<00:00, 2224.02it/s] 


In [24]:
size_num = runner.mon['N.spike'].shape[0]
print(size_num)
spike_calculate = runner.mon['N.spike'][size_num//4:,:]
sum_along_columns = np.sum(spike_calculate, axis=0)/total_period*4/3
print(sum_along_columns)

300000
[0.73466667 0.10044444 1.08488889]
